مرحبا


In [5]:
!sudo apt install poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [15]:
!wget -O ../documents/aws-cloud-adoption-framework.pdf -nc https://d1.awsstatic.com/whitepapers/International/ar/aws-cloud-adoption-framework.pdf 

--2024-08-07 16:33:50--  https://d1.awsstatic.com/whitepapers/International/ar/aws-cloud-adoption-framework.pdf
Resolving d1.awsstatic.com (d1.awsstatic.com)... 99.84.70.192, 2600:9000:2163:9000:19:2ae8:f748:f7e1, 2600:9000:2163:9e00:19:2ae8:f748:f7e1, ...
Connecting to d1.awsstatic.com (d1.awsstatic.com)|99.84.70.192|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 1336106 (1.3M) [application/pdf]
Saving to: ‘../documents/aws-cloud-adoption-framework.pdf’

../documents/aws-cl 100%[===================>]   1.27M  --.-KB/s    in 0.03s   

2024-08-07 16:33:50 (39.0 MB/s) - ‘../documents/aws-cloud-adoption-framework.pdf’ saved [1336106/1336106]



In [16]:
document = "../documents/aws-cloud-adoption-framework.pdf"

In [28]:
!pdfinfo {document}

Title:           إطار عمل اعتماد السحابة من AWS
Subject:         إطار عمل اعتماد السحابة من AWS
Keywords:        CAF, Adoption
Author:          Amazon Web Services
Creator:         Microsoft Word
CreationDate:    Fri Mar  4 21:17:33 2022 UTC
ModDate:         Fri Mar  4 21:35:12 2022 UTC
Custom Metadata: no
Metadata Stream: yes
Tagged:          yes
UserProperties:  no
Suspects:        no
Form:            none
JavaScript:      no
Pages:           25
Encrypted:       no
Page size:       612 x 792 pts (letter)
Page rot:        0
File size:       1336106 bytes
Optimized:       no
PDF version:     1.7


In [36]:
filename = raw_text_output_path.split("/")[-1].split(".")[0]
raw_text_output_path = f"../output/raw/{filename}.txt"
raw_text_output_path

'../output/raw/aws-cloud-adoption-framework.txt'

In [34]:
!pdftotext -layout {document} {raw_text_output_path}

In [35]:
!head {raw_text_output_path}

 ‫إطار عمل اعتماد السحابة من ‪AWS‬‬
‫التحول الرقمي المدعوم بالسحابة‬
                         ‫ّ‬     ‫تسريع عمليات‬



       ‫تم نشره ألول مرة في فبراير‪/‬شباط ‪2015‬‬

     ‫تم التحديث في ‪ 22‬نوفمبر‪ /‬تشرين الثاني ‪2021‬‬
                                                                                               ‫اإلشعارات‬


In [37]:
with open(raw_text_output_path) as txt:
    text_lines = txt.readlines()

In [39]:
text_lines[:10]

[' \u202bإطار عمل اعتماد السحابة من \u202aAWS\u202c\u202c\n',
 '\u202bالتحول الرقمي المدعوم بالسحابة\u202c\n',
 '                         \u202bّ\u202c     \u202bتسريع عمليات\u202c\n',
 '\n',
 '\n',
 '\n',
 '       \u202bتم نشره ألول مرة في فبراير\u202a/\u202cشباط \u202a2015\u202c\u202c\n',
 '\n',
 '     \u202bتم التحديث في \u202a 22\u202cنوفمبر\u202a /\u202cتشرين الثاني \u202a2021\u202c\u202c\n',
 '\x0c                                                                                               \u202bاإلشعارات\u202c\n']

In [42]:
import re


def clean_line(line):
    unicodes_to_replace = ["\u202a", "\u202b", "\u202c"]
    for unicode in unicodes_to_replace:
        line = line.replace(unicode, "")
    return line


pattern = re.compile(r'\s{10,}')
cleaned_text = ""

for line in text_lines:
    cleaned_line = clean_line(line).strip()
    result = pattern.split(cleaned_line)[::-1]

    if len(result) == 1:
        line = result[0]
        if line:
            cleaned_text += line + "\n"
        else:
            # handle empty lines
            cleaned_text += "\n"

    else:
        format_string = " ".join(["{: >50}"] * len(result))
        # handle multiple columns lines
        cleaned_text += format_string.format(*result) + "\n"
        # print(cleaned_text)
        # cleaned_text += "             ".join(result) + "\n"


cleaned_text_output_path = f"../output/cleaned/{filename}.txt"

with open(cleaned_text_output_path, "w") as txt:
    txt.write(cleaned_text)